Pre processing meteorological data

# Intro

## Libraries

In [1]:
import os
import glob
from datetime import datetime

import pandas as pd
import numpy as np
from decimal import Decimal
import collections
import time

from sklearn import datasets, linear_model
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from brokenaxes import brokenaxes

from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr

from functools import reduce

# from co2mpas.datasync import _get_interp_method

## Input files

In [2]:
# ESTACIONES

path1 = r'C:/CicloHidrico/'         # directorio de las estaciones
os.chdir(path1)

estID = pd.read_csv("estaciones.csv", sep=';', engine='python', index_col=0)                   # estaciones con ID number
estRang = pd.read_excel("EstacionesTermoPluvio.xlsx", header=1, sep=';', usecols=range(1, 10)) # estaciones con rango de fecha

# print(os.getcwd())
# # Display all of the files found in your current working directory
# print(os.listdir(os.getcwd()))

In [7]:
# DATOS PRECIPITACIÓN PRINCIPALES

path2 = r'C:/CicloHidrico/Datos'      # directorio de los datos
os.chdir(path2)

precipitacion = pd.read_csv("precipitacion.csv", sep=';',
                            engine='python', index_col=0)   # precipitacion total diaria DB

In [8]:
t_max = pd.read_csv("temperatura_max.csv", sep=';', index_col=0)      # temperaturas màximas diarias
t_max.describe()

,Tmaxima
count,3.508029e+06
mean,2.059618e+01
std,8.680685e+00
min,-9.900000e+01
25%,1.500000e+01
50%,2.100000e+01
75%,2.710000e+01
max,5.270000e+01


In [9]:
t_min = pd.read_csv("temperatura_min.csv", sep=';', index_col=0)      # temperaturas màximas diarias
t_min.describe()

,Tminima
count,3.507394e+06
mean,9.963253e+00
std,6.859695e+00
min,-9.900000e+01
25%,5.000000e+00
50%,1.000000e+01
75%,1.500000e+01
max,3.900000e+01


In [10]:
# DATOS PRECIPITACIÓN 2 (actualización/últimos datos) - CU, AB, TE de los últimos años

path3 = r'C:/CicloHidrico/AemetBernat'      # directorio de los datos
os.chdir(path3)

pp2 = pd.read_csv("PrecipitacionDiaria.csv", sep=';',
                            engine='python', index_col=0)   # precipitacion total diaria DB

In [11]:
pp2.describe(include='all')   

,AÑO,MES,NOMBRE,ALTITUD,NOM_PROV,LONGITUD,LATITUD,DATUM,P1,P2,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
count,82732.000000,82732.000000,82732,82732.000000,82732,82732.000000,82732.000000,82732,82350.000000,82385.000000,...,82403.000000,82397.000000,82374.000000,82377.000000,82368.000000,82398.000000,82351.000000,77112.000000,75332.000000,47873.000000
unique,NaN,NaN,144,NaN,3,NaN,NaN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,ONTUR GRUPO ESCOLAR,NaN,CUENCA,NaN,NaN,ETRS89,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,1124,NaN,41661,NaN,NaN,82732,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1984.692284,6.479367,NaN,988.529916,NaN,154242.784219,395487.459447,NaN,12.227747,15.139965,...,14.818211,12.976201,13.054617,13.918691,13.160014,15.235188,16.744083,16.155475,14.283492,14.791323
std,21.450645,3.454660,NaN,242.295165,NaN,58286.888399,7251.467824,NaN,45.468356,50.736757,...,51.905483,47.893030,47.416207,49.597009,45.637562,52.003338,55.761471,53.634729,51.017687,52.275781
min,1912.000000,1.000000,NaN,397.000000,NaN,8471.000000,380516.000000,NaN,-4.000000,-4.000000,...,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000,-3.000000
25%,1970.000000,3.000000,NaN,831.000000,NaN,125212.000000,391926.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1987.000000,6.000000,NaN,959.000000,NaN,151112.000000,395846.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2002.000000,9.000000,NaN,1150.000000,NaN,212262.000000,401555.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Pre processing

## Cambio formato datos PP2

In [14]:
pp2.index.unique()

Index(['3010', '3042', '3051', '3058', '3059', '3070', '3076', '4002', '4070',
       '4070Y',
       ...
       '9380', '9386A', '9531X', '9531Y', '9556', '9556U', '9557B', '9559',
       '9927', '9935A'],
      dtype='object', name='INDICATIVO', length=148)

## Datos de las estaciones

### Datos duplicados

In [8]:
"""
Estaciones con el nombre repetido, con el Código repetido o con las coordenadas geográficas repetidas
"""
# duplicateRowsDF = estID.Estacion[estID.duplicated(['Estacion'])==True]
# duplicateRowsDF = estID.Estacion[estID.duplicated(['CodigoEst'])==True]

duplicateRowsDF = estID.Estacion[estID.duplicated(['Altitud','Xutm','Yutm'])==True] 
print("Duplicate Rows except first occurrence based on geographic coordinates are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on geographic coordinates are :
260                            XABIA
263    XABIA, AJUNTAMENT, AUTOMATICA
306                       GANDIA, HE
307                           GANDIA
507            BUNYOL, C.P. SAN LUIS
513                      CARLET, SMN
552              ALDAIA, HORT NOTARI
613              CHELVA, EL CALVARIO
666                 MASSAMAGRELL, HS
683             SEGORBE, LA ARTELINA
711                  NULES, PIUS XII
723        VILA-REAL, CENTRO C. AGR.
832          CASTELLFORT, AUTOMATICA
Name: Estacion, dtype: object


### Unir DB estaciones

In [7]:
estIDabc=estID.sort_values('Estacion').reset_index(drop=True)                              # ordena estaciones alfabéticamente

estRang = estRang.rename(columns={'Longitud': 'Long', 'Latitud': 'Lat', 'Provincia':'Prov'}) # evita columnas con mismo nombre
estaciones = estIDabc.join(estRang)                                                            # concatena DFs

estaciones.head()

,CodigoEst,Estacion,Tipo,Longitud,Latitud,Altitud,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,ESTACIÓN,Long,Lat,Altura,Prov,Cobertura Temperatura,Cobertura Precipitación,Temp,Pluv
0,1387,A CORUNYA (ESTACION COMPLETA),INMman,#¡CAMPO!,43.37,58,59149,4791283,AC,1,A CORUNYA (ESTACION COMPLETA),-8.42,43.37,58,AC,<->,1930-10-01<->2010-10-31,NO,SI
1,8359,ABABUJ,INMman,#¡CAMPO!,40.56,1368,686597,4491216,TE,3,ABABUJ,-1.28,40.56,1368,TE,<->,<->,NO,NO
2,7250,"ABANILLA, CHS",INMman,#¡CAMPO!,38.20,222,672506,4229911,MU,2,"ABANILLA, CHS",-1.03,38.20,222,MU,1958-01-01<->2010-10-31,1958-01-01<->2010-10-31,SI,SI
3,7150,ABARAN (SIERRA DE LA PILA),INMman,#¡CAMPO!,38.25,300,646400,4235009,MU,3,ABARAN (SIERRA DE LA PILA),-1.32,38.25,300,MU,2007-10-01<->2010-10-31,1961-04-01<->2010-10-31,SI,SI
4,7151,ABARAN (SIERRA DEL ORO),INMman,#¡CAMPO!,38.18,400,638133,4227865,MU,3,ABARAN (SIERRA DEL ORO),-1.42,38.18,400,MU,2007-10-01<->2010-10-31,1957-04-01<->2010-10-31,SI,SI


## Merge DB

In [8]:
# une datos de las estaciones a la DB de precipitacion
dataPP = pd.merge(precipitacion,
                estaciones[['Estacion','CodigoEst','Tipo','Xutm','Yutm','Altura','Provincia','Umbral_20_40_km_costa']],
                on='CodigoEst')
dataPP.describe(include='all')

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa
count,9895905,9895905,9.895905e+06,9895905,9895905,9895905,9.895905e+06,9.895905e+06,9.895905e+06,9895905,9.895905e+06
unique,840,41641,NaN,3,835,4,NaN,NaN,NaN,29,NaN
top,9981A,1994-04-23,NaN,S,"TORTOSA, OBSER. DEL EBRO",INMman,NaN,NaN,NaN,V,NaN
freq,36012,507,NaN,9389160,36012,9734857,NaN,NaN,NaN,2717440,NaN
mean,NaN,NaN,-3.226154e+00,NaN,NaN,NaN,6.568422e+05,4.349168e+06,4.853115e+02,NaN,2.170715e+00
std,NaN,NaN,2.210382e+01,NaN,NaN,NaN,1.123354e+05,1.112732e+05,3.946691e+02,NaN,8.670858e-01
min,NaN,NaN,-9.990000e+01,NaN,NaN,NaN,3.344300e+04,4.075366e+06,1.000000e+00,NaN,1.000000e+00
25%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.209690e+05,4.270236e+06,8.600000e+01,NaN,1.000000e+00
50%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.913740e+05,4.343742e+06,4.190000e+02,NaN,2.000000e+00
75%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,7.226450e+05,4.418167e+06,8.100000e+02,NaN,3.000000e+00


In [13]:
# une las temperaturas máx y mín a la DB precipitaciones

from functools import reduce

data_raw = reduce(lambda x,y: pd.merge(x,y, on=['CodigoEst','Fecha'], how='outer'), [dataPP, t_max, t_min])

In [41]:
data_raw.describe(include='all')

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
count,10046338,10046338,9.895905e+06,9895905,9895905,9895905,9.895905e+06,9.895905e+06,9.895905e+06,9895905,9.895905e+06,3.508029e+06,3508029,3.507394e+06,3507394
unique,844,47603,NaN,3,835,4,NaN,NaN,NaN,29,NaN,NaN,2,NaN,2
top,3195,1994-09-16,NaN,S,"TORTOSA, OBSER. DEL EBRO",INMman,NaN,NaN,NaN,V,NaN,NaN,S,NaN,S
freq,42958,511,NaN,9389160,36012,9734857,NaN,NaN,NaN,2717440,NaN,NaN,3293416,NaN,3306807
mean,NaN,NaN,-3.226154e+00,NaN,NaN,NaN,6.568422e+05,4.349168e+06,4.853115e+02,NaN,2.170715e+00,2.059618e+01,NaN,9.963253e+00,NaN
std,NaN,NaN,2.210382e+01,NaN,NaN,NaN,1.123354e+05,1.112732e+05,3.946691e+02,NaN,8.670858e-01,8.680685e+00,NaN,6.859695e+00,NaN
min,NaN,NaN,-9.990000e+01,NaN,NaN,NaN,3.344300e+04,4.075366e+06,1.000000e+00,NaN,1.000000e+00,-9.900000e+01,NaN,-9.900000e+01,NaN
25%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.209690e+05,4.270236e+06,8.600000e+01,NaN,1.000000e+00,1.500000e+01,NaN,5.000000e+00,NaN
50%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.913740e+05,4.343742e+06,4.190000e+02,NaN,2.000000e+00,2.100000e+01,NaN,1.000000e+01,NaN
75%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,7.226450e+05,4.418167e+06,8.100000e+02,NaN,3.000000e+00,2.710000e+01,NaN,1.500000e+01,NaN


## Limpieza de datos

### Días sin precipitación

In [42]:
data_halfraw = data_raw.dropna(subset=['Precipitacion','ValidPP']) # elimina las filas que no tienen datos de precipitacion
data_halfraw.describe()

,Precipitacion,Xutm,Yutm,Altura,Umbral_20_40_km_costa,Tmaxima,Tminima
count,9.895905e+06,9.895905e+06,9.895905e+06,9.895905e+06,9.895905e+06,3.357939e+06,3.357324e+06
mean,-3.226154e+00,6.568422e+05,4.349168e+06,4.853115e+02,2.170715e+00,2.061929e+01,1.000101e+01
std,2.210382e+01,1.123354e+05,1.112732e+05,3.946691e+02,8.670858e-01,8.653267e+00,6.855328e+00
min,-9.990000e+01,3.344300e+04,4.075366e+06,1.000000e+00,1.000000e+00,-9.900000e+01,-9.900000e+01
25%,0.000000e+00,6.209690e+05,4.270236e+06,8.600000e+01,1.000000e+00,1.500000e+01,5.000000e+00
50%,0.000000e+00,6.913740e+05,4.343742e+06,4.190000e+02,2.000000e+00,2.100000e+01,1.000000e+01
75%,0.000000e+00,7.226450e+05,4.418167e+06,8.100000e+02,3.000000e+00,2.710000e+01,1.510000e+01
max,8.780000e+02,8.150770e+05,4.828087e+06,1.890000e+03,3.000000e+00,5.270000e+01,3.810000e+01


### Datos de precipitación NO válidos

In [43]:
data_halfraw.isnull().sum()    # no NaN data in Precipitacion, only in Temperatures

CodigoEst                      0
Fecha                          0
Precipitacion                  0
ValidPP                        0
Estacion                       0
Tipo                           0
Xutm                           0
Yutm                           0
Altura                         0
Provincia                      0
Umbral_20_40_km_costa          0
Tmaxima                  6537966
ValidTmax                6537966
Tminima                  6538581
ValidTmin                6538581
dtype: int64

In [51]:
data_halfraw.ValidPP.unique()                              # dice que en ValidPP, a parte de S y N, hay un campo A...

array(['S', 'N', 'A'], dtype=object)

In [55]:
# Input
ValidPP = ['N', 'A']          # casos con validez diferente a 'S'
value = [-99.9, -0.4,-0.3, 0] # valores concretos con precipitación menor que 0

print ('--- Validez de la precipitación ---')
print("\n")

for each1 in ValidPP:
    lentot = len(data_halfraw[(data_halfraw.ValidPP == each1)])
    lenpos = len(data_halfraw[(data_halfraw.ValidPP == each1) & (data_halfraw.Precipitacion > 0)])
    print ('Hay '+str(lentot)+' casos con validez '+ each1+' , de los cuales '+str(lenpos) +' tienen precipitacion > 0')
    for each2 in value:
        lenneg = len(data_halfraw[(data_halfraw.ValidPP == each1) & (data_halfraw.Precipitacion == each2)])
        print('    Cuando la validez es '+ each1 +' y la cantidad es ' + str(each2)+' mm, hay '+ str(lenneg) +' casos' )
    print("\n")

--- Validez de la precipitación ---


Hay 0 casos con validez N , de los cuales 0 tienen precipitacion > 0
    Cuando la validez es N y la cantidad es -99.9 mm, hay 0 casos
    Cuando la validez es N y la cantidad es -0.4 mm, hay 0 casos
    Cuando la validez es N y la cantidad es -0.3 mm, hay 0 casos
    Cuando la validez es N y la cantidad es 0 mm, hay 0 casos


Hay 8020 casos con validez A , de los cuales 6613 tienen precipitacion > 0
    Cuando la validez es A y la cantidad es -99.9 mm, hay 0 casos
    Cuando la validez es A y la cantidad es -0.4 mm, hay 1033 casos
    Cuando la validez es A y la cantidad es -0.3 mm, hay 3 casos
    Cuando la validez es A y la cantidad es 0 mm, hay 371 casos




In [56]:
# elimina validez N (linea 1) y elimina validez A solo en el caso que pp = -99.9 mm 

data_halfraw = data_halfraw.drop(data_halfraw[(data_halfraw['ValidPP'] == 'N') |    
                                (data_halfraw['ValidPP'] == 'A') & (data_halfraw.Precipitacion == (-99.9))].index)
len(data_raw)-len(data_halfraw)

649158

### Datos de temperatura NO válidos

In [20]:
# elimina SOLO las temperaturas cuando el dato no es válido, deja el resto de la fila

data_halfraw.Tmaxima[data_halfraw['ValidTmax'] == 'N'] = np.nan
data_halfraw.Tminima[data_halfraw['ValidTmin'] == 'N'] = np.nan

C:\Users\ceam\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ceam\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


### Datos de fecha fuera de rango

In [21]:
# con la anterior limpieza (por ValidPP), ha desaparecido también los errores de fecha (out of range)

len(data_halfraw[data_halfraw.Fecha>'2018-12-31'])                     # datos en 2019: 1805, todos de enero
len(data_halfraw.CodigoEst[data_halfraw.Fecha>'2018-12-31'].unique())  # número de estaciones con datos en 2019: 60

len(data_halfraw[(data_halfraw.Fecha<'1900-01-01') & (data_halfraw.Fecha>'2019-06-01')]) # no data out of range

0

In [58]:
# número de estaciones cuyos datos fueron todos eliminados por validez de precipitación

a = data_halfraw.CodigoEst.unique() # estaciones con datos válidos
b = data_raw.CodigoEst.unique()     # estaciones con datos

main_list = np.setdiff1d(b,a)
main_list                            # lista de estaciones sin datos válidos (8096 = Cuenca)

#len(b)-len(a)                     #  número estaciones sin datos válidos

array(['7028I', '7138A', '8008A', '8096', '8325C'], dtype=object)

### Formatos uniformes

In [61]:
# longitud de cada uno de los items de una Serie (columna). para saber si uniforme, usar mean() o mejor, describe()

data_halfraw.Fecha.str.len().describe()

count    9397180.0
mean          10.0
std            0.0
min           10.0
25%           10.0
50%           10.0
75%           10.0
max           10.0
Name: Fecha, dtype: float64

In [74]:
data_halfraw.Xutm.astype('str').str.len().describe()    # num de dígitos en Xutm

count    8661698.0
mean           8.0
std            0.0
min            8.0
25%            8.0
50%            8.0
75%            8.0
max            8.0
Name: Xutm, dtype: float64

In [63]:
data_halfraw.Yutm.astype('str').str.len().describe()    # num de dígitos en Yutm

count    9397180.0
mean           9.0
std            0.0
min            9.0
25%            9.0
50%            9.0
75%            9.0
max            9.0
Name: Yutm, dtype: float64

In [76]:
data_halfraw.tail()

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
3562254,8073,2018-11-26,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
3562239,8073,2018-11-27,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
3562224,8073,2018-11-28,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
3562210,8073,2018-11-29,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
3562196,8073,2018-11-30,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN


In [77]:
data_halfraw.dtypes                                     # formato de las columnas de mi DB

CodigoEst                 object
Fecha                     object
Precipitacion            float64
ValidPP                   object
Estacion                  object
Tipo                      object
Xutm                     float64
Yutm                     float64
Altura                   float64
Provincia                 object
Umbral_20_40_km_costa    float64
Tmaxima                  float64
ValidTmax                 object
Tminima                  float64
ValidTmin                 object
dtype: object

### Estaciones fuera de rango geogràfico

In [65]:
# Eliminar datos de estaciones que no sean C, V, A, MU, AB, CU, TE

data_halfraw = data_halfraw[data_halfraw.Provincia.isin(['CU', 'AB','MU','TE','C','V','A'])]

### Ordenar

In [78]:
### La jerarquía es: 1. Provincia, 2. Nombre de estación, 3. Fecha

data_halfraw.sort_values(by=['Provincia','Estacion','Fecha'], inplace=True)

In [79]:
data_halfraw.head()

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
979498,8021A,1975-05-01,0.0,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,376.0,A,1.0,NaN,NaN,NaN,NaN
979499,8021A,1975-05-02,0.0,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,376.0,A,1.0,NaN,NaN,NaN,NaN
979500,8021A,1975-05-03,0.0,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,376.0,A,1.0,NaN,NaN,NaN,NaN
979501,8021A,1975-05-04,2.8,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,376.0,A,1.0,NaN,NaN,NaN,NaN
979502,8021A,1975-05-05,15.9,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,376.0,A,1.0,NaN,NaN,NaN,NaN


# To Excel

In [80]:
meteodata = data_halfraw
del data_halfraw

In [81]:
# reset index per a que estiga ordenat de 0 a X
meteodata.reset_index(drop=True, inplace=True)

In [83]:
meteodata.to_csv('meteodata.csv')

## Other preprocessing

In [3]:
path2 = r'C:/CicloHidrico/Datos'      # directorio de los datos
os.chdir(path2)

In [4]:
data = pd.read_csv("meteodata.csv", index_col=0) 

C:\Users\ceam\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\ceam\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
#adding Year Month Day columns

data['Year']=[d.split('-')[0] for d in data.Fecha]
data['Month']=[d.split('-')[1] for d in data.Fecha]
data['Day']=[d.split('-')[2] for d in data.Fecha]

data.tail(5)

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin,Year,Month,Day
8661693,8073,2018-11-26,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,26
8661694,8073,2018-11-27,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,27
8661695,8073,2018-11-28,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,28
8661696,8073,2018-11-29,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,29
8661697,8073,2018-11-30,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,30


In [6]:
data.CodigoEst = data.CodigoEst.astype('str')

In [7]:
len(data.CodigoEst.unique())

803

In [8]:
est = pd.read_excel("estCHjuc.xlsx",index_col=0)  # con las dist_costa

In [9]:
len(est)

803

In [10]:
data = data.merge(est[['CodigoEst','dist_costa']],on='CodigoEst',how='left').reset_index(drop=True).drop_duplicates()

In [11]:
data.dist_costa.count()

8661698

In [13]:
data.to_csv('meteodata.csv')